![alt text](https://)This code was orginally found here by the tensorflow team:
https://github.com/tensorflow/tpu/tree/master/models/samples/core/get_started

Please fill out the values in the box below before running the whole notebook!
Also, this will either work as a .py file or as a notebook in collab.

Creates the directory for the model be saved and checks for the directory

In [58]:
!mkdir irisdata
!ls

mkdir: cannot create directory ‘irisdata’: File exists
adc.json  irisdata  sample_data


Parameter setup

In [59]:
import time 
import json
import pprint

use_tpu = True #@param {type:"boolean"}
bucket = 'sweetc' #@param {type:"string"}

assert bucket, 'Must specify an existing GCS bucket name'
print('Using bucket: {}'.format(bucket))


model_dir = 'gs://{}/{}'.format(bucket, time.strftime('tpuestimator-dnn/%Y-%m-%d-%H-%M-%S'))
print('Using model dir: {}'.format(model_dir))

from google.colab import auth
auth.authenticate_user()

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])

  # Upload credentials to TPU.
  with tf.Session(TF_MASTER) as sess:
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
else:
  TF_MASTER=''

with tf.Session(TF_MASTER) as session:
  print ('List of devices:')
  pprint.pprint(session.list_devices())

Using bucket: sweetc
Using model dir: gs://sweetc/tpuestimator-dnn/2020-03-09-23-31-58
List of devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 17538401081182207055),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3395095972937578274),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 874212400741116587),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8680662906555072639),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 12331013487340557674),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 15652422744169196498),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 15706229582512420389),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 4041186063180486278),
 _DeviceAttributes(/job:tpu_worker/replica:0/t

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.contrib import cluster_resolver as contrib_cluster_resolver
from tensorflow.contrib import tpu as contrib_tpu
import pandas as pd
import tensorflow.compat.v1 as tf
import os

# Cloud TPU Cluster Resolver flags
'''
The Cloud TPU to use for training. This should be the name used when 
creating the Cloud TPU. To find out hte name of TPU, either use command 
gcloud compute tpus list --zone=<zone-name>', or use 
ctpu status --details' if you have created Cloud TPU using ctpu up.
This code is for collab, to change it to work locally, use the IP or name where
the os.environ is located. 
'''

tpu = 'grpc://' + os.environ['COLAB_TPU_ADDR']

# Model specific parameters
'''
This should be the path of GCS bucket or a local directory which will be used as 
model_directory to export the checkpoints during training.
'''

#model_dir = 'gs://sweetc'

'''
This is the global batch size and not the per-shard batch.
'''
batch_size=8

'''
Total number of training steps.
'''
train_steps=1000

# TPU specific parameters.
'''
True, if want to run the model on TPU. False, otherwise.
'''
use_tpu=True

'''
Number of iterations per TPU training loop.
'''
iterations = 500

Set up for data download 

In [0]:
TRAIN_URL = 'http://download.tensorflow.org/data/iris_training.csv'
TEST_URL = 'http://download.tensorflow.org/data/iris_test.csv'

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

Download data

In [0]:
def download_path():
  train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
  test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

  return train_path, test_path


def load_data(y_name='Species'):
  """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
  train_path, test_path = download_path()

  train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0,
                      dtype={'SepalLength': pd.np.float32,
                             'SepalWidth': pd.np.float32,
                             'PetalLength': pd.np.float32,
                             'PetalWidth': pd.np.float32,
                             'Species': pd.np.int32})
  train_x, train_y = train, train.pop(y_name)

  test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0,
                     dtype={'SepalLength': pd.np.float32,
                            'SepalWidth': pd.np.float32,
                            'PetalLength': pd.np.float32,
                            'PetalWidth': pd.np.float32,
                            'Species': pd.np.int32})
  test_x, test_y = test, test.pop(y_name)

  return (train_x, train_y), (test_x, test_y)

Data input and metric functions

In [0]:
def train_input_fn(features, labels, batch_size):
  """An input function for training."""

  # Convert the inputs to a Dataset.
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  # Shuffle, repeat, and batch the examples.
  dataset = dataset.shuffle(1000).repeat()

  dataset = dataset.batch(batch_size, drop_remainder=True)

  # Return the dataset.
  return dataset

def metric_fn(labels, logits):
    """Function to return metrics for evaluation."""

    predicted_classes = tf.argmax(logits, 1)
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name="acc_op")
    return {"accuracy": accuracy}

Model 

In [0]:
def my_model(features, labels, mode, params):
    """Deep Neural Network(DNN) model.
    This is a DNN Model with 3 hidden layers. First 2 hidden layers are having
    10 neurons in each. And number of neurons in the last layer is equal to the
    number of output classes. This is a densely connected network where each
    neuron of previous layer is connected to each neuron of next layer.
    Args:
      features: Feature values for input samples.
      labels: label/class assigned to the corresponding input sample.
      mode: "TRAIN"/"EVAL"/"PREDICT"
      params: Dictionary used to pass extra parameters to model function from
        the main function.
    Returns:
      TPUEstimator object.
    """

    # Create three fully connected layers.
    net = tf.feature_column.input_layer(features, params["feature_columns"])
    for units in params["hidden_units"]:
      net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params["n_classes"], activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
      predictions = {
          "class_ids": predicted_classes[:, tf.newaxis],
          "probabilities": tf.nn.softmax(logits),
          "logits": logits,
      }
      return contrib_tpu.TPUEstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,
                                                  logits=logits)

    if mode == tf.estimator.ModeKeys.EVAL:
      return contrib_tpu.TPUEstimatorSpec(
          mode=mode, loss=loss, eval_metrics=(metric_fn, [labels, logits]))

    # Create training op.
    if mode == tf.estimator.ModeKeys.TRAIN:
      optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
      if use_tpu:
        optimizer = contrib_tpu.CrossShardOptimizer(optimizer)
      train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
      return contrib_tpu.TPUEstimatorSpec(mode, loss=loss, train_op=train_op)

Main code to run the whole thing 

In [65]:
def main(argv):
    del argv  # Unused

    # Fetch the data
    (train_x, train_y), (test_x, test_y) = load_data()

    # Feature columns describe how to use the input.
    my_feature_columns = []
    for key in train_x.keys():
      my_feature_columns.append(tf.feature_column.numeric_column(key=key))

    # Resolve TPU cluster and runconfig for this.
    tpu_cluster_resolver = contrib_cluster_resolver.TPUClusterResolver(tpu)

    run_config = contrib_tpu.RunConfig(
        model_dir=model_dir,
        cluster=tpu_cluster_resolver,
        session_config=tf.ConfigProto(
            allow_soft_placement=True, log_device_placement=True),
        tpu_config=contrib_tpu.TPUConfig(iterations),
    )

    # Build 2 hidden layer DNN with 10, 10 units respectively.
    classifier = contrib_tpu.TPUEstimator(
        model_fn=my_model,
        use_tpu=use_tpu,
        train_batch_size=batch_size,
        eval_batch_size=batch_size,
        predict_batch_size=batch_size,
        config=run_config,
        params={
            # Name of the feature columns in the input data.
            "feature_columns": my_feature_columns,
            # Two hidden layers of 10 nodes each.
            "hidden_units": [10, 10],
            # The model must choose between 3 classes.
            "n_classes": 3,
            "use_tpu": use_tpu,
        })

    # Train the Model.
    classifier.train(
        input_fn=lambda params: train_input_fn(
            train_x, train_y, params["batch_size"]),
        max_steps=train_steps)

    #To eval and prediction, use a smaller tpu such as v3-8. 

if __name__ == "__main__":
  tf.logging.set_verbosity(tf.logging.INFO)
  tf.app.run(main)


INFO:tensorflow:Using config: {'_model_dir': 'gs://sweetc/tpuestimator-dnn/2020-03-09-23-31-58', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
log_device_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.90.92.50:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1a4c9d0ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.90.92.50:8470', '_evaluation_master': 'grpc://10.90.92.50:8470', '_is_chief': True, '_num

I0309 23:31:59.965969 139752863999872 estimator.py:212] Using config: {'_model_dir': 'gs://sweetc/tpuestimator-dnn/2020-03-09-23-31-58', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
log_device_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.90.92.50:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1a4c9d0ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.90.92.50:8470', '_evaluation_master': 'grpc://10.

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0309 23:31:59.968578 139752863999872 tpu_context.py:220] _TPUContext: eval_on_tpu True


INFO:tensorflow:Querying Tensorflow master (grpc://10.90.92.50:8470) for TPU system metadata.


...

I0309 23:32:31.696572 139752863999872 estimator.py:371] Loss for final step: 0.0032541663.


INFO:tensorflow:training_loop marked as finished


I0309 23:32:31.698261 139752863999872 error_handling.py:101] training_loop marked as finished


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
